In [1]:
from encodec import EncodecModel
from encodec.utils import convert_audio

import torchaudio
import torch
from encodec.compress import compress, decompress, decompress_from_frames

In [13]:
model = EncodecModel.encodec_model_24khz()
model.set_target_bandwidth(3)

In [14]:
# Load and pre-process the audio waveform
wav, sr = torchaudio.load("/net/papilio/storage2/yhaoyuan/LAbyLM/audios_trimmed_select/3/94.wav")
wav = wav.unsqueeze(0)
wav = convert_audio(wav, sr, model.sample_rate, model.channels)
#print(wav.shape)
#compressed = compress(model, wav, use_lm = False)
# Extract discrete codes from EnCodec
with torch.no_grad():
    encoded_frames = model.encode(wav)
codes = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1)  # [B, n_q, T]

In [15]:
codes.shape

torch.Size([1, 4, 356])

In [16]:
codes

tensor([[[ 62, 408, 408,  ...,  62,  62,  62],
         [913, 518, 518,  ..., 518, 424, 424],
         [829, 863, 786,  ..., 786,  36, 786],
         [673, 673, 741,  ..., 215, 673, 673]]])

In [17]:
wav, sr = decompress_from_frames(model, codes)
wav.numpy()

array([[ 0.00038799, -0.00011974,  0.00023662, ...,  0.00028873,
         0.00030056,  0.00031663]], dtype=float32)

In [18]:
import IPython.display as ipd
display(ipd.Audio(wav, rate = 24000))